In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pydataset import data 
print('Success')

Success


# Tips

In [57]:
tips = data("tips")
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


* Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [58]:
tips['tip_percentage'] = tips['tip']/tips['total_bill'] * 100
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,5.944673
2,10.34,1.66,Male,No,Sun,Dinner,3,16.054159
3,21.01,3.50,Male,No,Sun,Dinner,3,16.658734
4,23.68,3.31,Male,No,Sun,Dinner,2,13.978041
5,24.59,3.61,Female,No,Sun,Dinner,4,14.680765


* Create a column named price_per_person. This should be the total bill divided by the party size.

In [59]:
tips['price_per_person'] = tips['total_bill']/tips['size']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,5.944673,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,16.054159,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,16.658734,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,13.978041,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,14.680765,6.147500


* Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

* ***Answer*** - Tip Amount = Total Bill, Tip Percentage = Party Size

* Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

# K Best - Tips

In [60]:
tips = pd.get_dummies(tips)
tips.drop(columns=['sex_Female', 'smoker_No'], inplace = True)

In [61]:
from sklearn.model_selection import train_test_split

train_validate, test = train_test_split(tips, train_size=0.8, random_state=222)
train, validate = train_test_split(train_validate, train_size=0.75, random_state=222)

# Create the scaler and fit it to the training dataset.
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler(copy=True).fit(train)

# Scale the training data
train = pd.DataFrame(scaler.transform(train), columns=train.columns.values).set_index([train.index.values])

# Set aside the validate and test datasets for our "out of sample" data
validate = pd.DataFrame(scaler.transform(validate), columns=validate.columns.values).set_index([validate.index.values])
test = pd.DataFrame(scaler.transform(test), columns=test.columns.values).set_index([test.index.values])

X_train = train.drop(columns=['tip'])
y_train = train[['tip']]

X_validate = validate.drop(columns='tip')
y_validate = validate[['tip']]

X_train.head(1)

,total_bill,size,tip_percentage,price_per_person,sex_Male,smoker_Yes,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch
234,0.170354,0.2,0.122428,0.134554,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [62]:
from sklearn.feature_selection import SelectKBest, f_regression

# Create the f_selector object 
f_selector = SelectKBest(f_regression, k=2)

# fit the selector to the training data
f_selector.fit(X_train, y_train)

# Reduce the training data by transforming the X_train data set to the f_selector
X_reduced = f_selector.transform(X_train)

# Use get support to create a mask
f_support = f_selector.get_support()

f_feature = X_train.loc[:,f_support].columns.tolist()

print(str(len(f_feature)), 'selected features')
print(f_feature)

2 selected features
['total_bill', 'size']


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Recursive Feature - Tips

In [63]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# Use linear regressions learning model object 
lm = LinearRegression()

# Create the recursive feature test using the LM and selecting the top two features
rfe = RFE(lm, 2)

# Create X_rfe object by fitting and transforming the X_train, y_train data to X_rfe 
X_rfe = rfe.fit_transform(X_train,y_train)  

# then fit the object to the learning model
lm.fit(X_rfe,y_train)

# Create the mask to get test the results against the training model in order to determine which features...
# are being used 

mask = rfe.support_

# Create the features into  the list
rfe_features = X_train.loc[:,mask].columns.tolist()

# Pring the a list of the features
print(str(len(rfe_features)), 'selected features')
print(rfe_features)

2 selected features
['tip_percentage', 'price_per_person']


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


* Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?

# K Best - Tips_Percentage

In [64]:
X_train = train.drop(columns=['tip_percentage'])
y_train = train[['tip_percentage']]

X_validate = validate.drop(columns='tip_percentage')
y_validate = validate[['tip_percentage']]

# Create the f_selector object 
f_selector = SelectKBest(f_regression, k=2)

# fit the selector to the training data
f_selector.fit(X_train, y_train)

# Reduce the training data by transforming the X_train data set to the f_selector
X_reduced = f_selector.transform(X_train)

# Use get support to create a mask
f_support = f_selector.get_support()

f_feature = X_train.loc[:,f_support].columns.tolist()

print(str(len(f_feature)), 'selected features')
print(f_feature)

2 selected features
['total_bill', 'tip']


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Recursive Feature Elimination - Tips_Percentage

In [65]:
# Use linear regressions learning model object 
lm = LinearRegression()

# Create the recursive feature test using the LM and selecting the top two features
rfe = RFE(lm, 2)

# Create X_rfe object by fitting and transforming the X_train, y_train data to X_rfe 
X_rfe = rfe.fit_transform(X_train,y_train)  

# then fit the object to the learning model
lm.fit(X_rfe,y_train)

# Create the mask to get test the results against the training model in order to determine which features...
# are being used 

mask = rfe.support_

# Create the features into  the list
rfe_features = X_train.loc[:,mask].columns.tolist()

# Pring the a list of the features
print(str(len(rfe_features)), 'selected features')
print(rfe_features)

2 selected features
['tip', 'price_per_person']


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


* Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

* ***Answer*** 
    * Using rfe conducts several model test and selects the best one through consensus while K best only looks at the features on a whole set and selects the best from it
    * Yes, but only down to a point as the more features you choose, the greather the collective impact and therefore the less likely different features will be used.

* Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [66]:
def select_kbest(y, k):
    X_train = train.drop(columns=[y])
    y_train = train[[y]]

    # Create the f_selector object 
    f_selector = SelectKBest(f_regression, k=k)

    # fit the selector to the training data
    f_selector.fit(X_train, y_train)

    # Reduce the training data by transforming the X_train data set to the f_selector
    X_reduced = f_selector.transform(X_train)

    # Use get support to create a mask
    f_support = f_selector.get_support()

    f_feature = X_train.loc[:,f_support].columns.tolist()

    print(str(len(f_feature)), 'selected features') 
    print(f_feature)

select_kbest(tip, 2)

NameError: name 'tip' is not defined

* Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [69]:
def rfe(y, k):
    X_train = train.drop(columns=[y])
    y_train = train[[y]]
    
    # Use linear regressions learning model object 
    lm = LinearRegression()

    # Create the recursive feature test using the LM and selecting the top two features
    rfe = RFE(lm, k)

    # Create X_rfe object by fitting and transforming the X_train, y_train data to X_rfe 
    X_rfe = rfe.fit_transform(X_train,y_train)  

    # then fit the object to the learning model
    lm.fit(X_rfe,y_train)

    # Create the mask to get test the results against the training model in order to determine which features...
    # are being used 

    mask = rfe.support_

    # Create the features into  the list
    rfe_features = X_train.loc[:,mask].columns.tolist()

    # Pring the a list of the features
    print(str(len(rfe_features)), 'selected features')
    print(rfe_features)
    
rfe('tip_percentage', 2)

2 selected features
['tip', 'price_per_person']


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


* Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [49]:
swiss = data("swiss")
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [70]:
train_validate, test = train_test_split(swiss, train_size=0.8, random_state=222)
train, validate = train_test_split(train_validate, train_size=0.75, random_state=222)

# Create the scaler and fit it to the training dataset.

scaler = MinMaxScaler(copy=True).fit(train)

# Scale the training data
train = pd.DataFrame(scaler.transform(train), columns=train.columns.values).set_index([train.index.values])

# Set aside the validate and test datasets for our "out of sample" data
validate = pd.DataFrame(scaler.transform(validate), columns=validate.columns.values).set_index([validate.index.values])
test = pd.DataFrame(scaler.transform(test), columns=test.columns.values).set_index([test.index.values])

train.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
La Vallee,0.336237,0.158192,0.823529,0.365385,0.000000,0.000000
Glane,1.000000,0.752542,0.323529,0.134615,0.970976,0.892405
Paysd'enhaut,0.644599,0.703955,0.088235,0.038462,0.004190,0.455696
Orbe,0.390244,0.597740,0.500000,0.096154,0.020950,0.284810
Lavaux,0.524390,0.811299,0.470588,0.153846,0.007052,0.582278


In [71]:
rfe('Fertility', 3)

3 selected features
['Examination', 'Education', 'Infant.Mortality']


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
